In [307]:
import serial
import serial.tools.list_ports
import numpy as np
from datetime import datetime

In [ ]:
ports = serial.tools.list_ports.comports(include_links=False)

if ports:
    PORT = ports[0].device

print(PORT)

COM15


In [313]:
class FPGA_communication:
        def __init__(self, port, baud_rate):
            self.port = port
            self.baud_rate = baud_rate
        
        def open_instrument(self):
            self.ser = serial.Serial(self.port, self.baud_rate, timeout=1)
            self.log = open("log_fpga_communication.txt","a+")
            current_time = datetime.now()
            self.log.write(f"[INFO] - - - {current_time} - - - \n")
            print(f"[INFO] FPGA initialized on port {self.port}, baud rate: {self.baud_rate}")
            self.log.write(f"[INFO] FPGA initialized on port {self.port}, baud rate: {self.baud_rate}\n")

        def set_memory_addr(self):
            send = self.ser.write(b"A00")
            print("> bytes sent :" + str(send))
            self.log.write(f"[DEBUG] Sending memory adress\n")

            ret = self.ser.read(4)
            print("> return value : " + str(ret))
            if (ret == b"\x00OK\n"  or ret == b"\nOK\n"):
                print("> Set memory done")
                return True
            else:
                print("> Error in set memory")
                return False
            
        def write_val_mem(self, message:str):
             send = self.ser.write(b"W" + bytes(message, "utf-8"))
             print("> bytes sent :" + str(send))
             ret = self.ser.read(4)
             print("> return value : " + str(ret))
             if (ret == b"\nOK\n"):
                print("> Message sent")
                return True
             else:
                print("> Error in message sending")
                return False
             
        def display_mem_vals_leds(self):
            send = self.ser.write(b"G")
            print("> bytes sent :" + str(send))
            ret = self.ser.read(4)
            print("> return value : " + str(ret))
            if (ret == b"\nOK\n"):
                print("> Go sent")
                return True
            else:
                print("> Error in Go sending")
                return False
            
        def read_mem(self):
            send = self.ser.write(b"R")
            print("> bytes sent :" + str(send))
            ret = self.ser.read(5)
            print("> return value : " + str(ret))
            return ret.hex()[2]+ret.hex()[3]

        def close_instrument(self):
            self.ser.close() 

In [314]:
if __name__ == '__main__' and ports:
    fpga = FPGA_communication(port = PORT, baud_rate = 115200)
    fpga.open_instrument()
    fpga.set_memory_addr()
    fpga.write_val_mem("55")
    fpga.display_mem_vals_leds()
    mem_val = fpga.read_mem()
    print(mem_val)
    fpga.close_instrument()

[INFO] FPGA initialized on port COM15, baud rate: 115200
> bytes sent :3
> return value : b'\nOK\n'
> Set memory done
> bytes sent :3
> return value : b'\nOK\n'
> Message sent
> bytes sent :1
> return value : b'\nOK\n'
> Go sent
> bytes sent :1
> return value : b'\nUOK\n'
55
